In [1]:
from rasterstats import zonal_stats
import numpy as np
import pandas as pd
from osgeo import ogr

In [2]:
shapefile = 'shapefile/wards-river.shp'

In [3]:
# Read the shapefile and pull attributes from it into the data frame

driver = ogr.GetDriverByName("ESRI Shapefile")
dataSource = driver.Open(shapefile, 1)
layer = dataSource.GetLayer()

WARD = []
AREA = []
BOROUGH = []
NONLD_AREA = []
for feature in layer:
    WARD = WARD + [feature.GetField("NAME")]
    AREA = AREA + [feature.GetField("HECTARES")]
    BOROUGH = BOROUGH + [feature.GetField("BOROUGH")]
    NONLD_AREA = NONLD_AREA + [feature.GetField("NONLD_AREA")]

Because we subtracted the river, the true area of the polygon = Area-NONLDarea

In [4]:
df = pd.DataFrame(list(zip(WARD, BOROUGH, AREA, NONLD_AREA)), columns=["Ward", "Borough", "Area", 'NonLDArea'])
b = df['Area']-df['NonLDArea']
df['AreaWard(HA)'] = b 
df.drop(columns = ['Area', 'NonLDArea'], inplace = True)
print(df.head(5))

                     Ward               Borough  AreaWard(HA)
0       Chessington South  Kingston upon Thames       755.173
1  Tolworth and Hook Rise  Kingston upon Thames       259.464
2              Berrylands  Kingston upon Thames       145.390
3               Alexandra  Kingston upon Thames       268.506
4                Beverley  Kingston upon Thames       187.821


reference in the DEMs

In [5]:
DEM2014='DEM@50/DEM2014.tif'

definition to count the total footprint of buildings in a ward

In [6]:
def footprint(x):
        return (np.sum(x > 2))/4

In [7]:
def coverage(x):
        return (np.sum(x > -1))/4

In [8]:
def volume(x):
    return np.sum(x)/4 

In [9]:
def calc(data, year, master):
    stats=zonal_stats(shapefile, data, stats = ["count"], add_stats={f"F_{year}" : footprint, f"V_{year}" : volume, f"C_{year}" : coverage})
    df1=pd.DataFrame.from_dict(stats)
    master=pd.concat([master, df1], axis=1)
    master.drop(columns=["count"], inplace=True)
    return master

execution

In [10]:
%%time
df=calc(DEM2014, 2014, df)

CPU times: user 2min 58s, sys: 5min 52s, total: 8min 51s
Wall time: 9min 4s


In [11]:
df.to_csv("data_pull_2014@50.csv")